In [1]:
import os
import re
import json
import pprint
import random

import pandas as pd
from collections import Counter

import spacy
from spacy.tokens import Doc
from spacy.gold import GoldParse
from spacy.language import EntityRecognizer
from spacy.util import minibatch, compounding

from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
nlp = spacy.load("en_core_web_lg") 

if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")
    
stripPersonnas = True
replace = {
    'rick sanches': 'rick',
    'drunk rick': 'rick',
    'morty smith': 'morty',
    'gromfomite': 'gromflomite',
    'other gromflomite': 'gromflomite',
    'gromflomite captain': 'gromflomite',
    'gromflomite worker': 'gromflomite',
    'gromflomite guard': 'gromflomite',
    'gromflomite office employee': 'gromflomite',
    'rick salseman': 'rick salesman',
    'armed ricks': 'armed rick',
    'all religious mortys': 'religious morty',
    'little tommy': 'tommy',
    'meeseek': 'meeseeks',
    'all the meeseeks': 'meeseeks',
    'all other meeseeks': 'meeseeks',
    'mailmen': 'mailman',
    'gerry': 'jerry'
    
}

ignore = [
    'a mexican armada shows up. with weapons made from two', 
    'pa system', 
    'all classmates except morty',
    'another voice',
    'sexualized s',
    'both']

In [2]:
json_data = None
characters = None
with open('data.json') as json_file:
    json_data = json.load(json_file)
    characters = json_data['characters'].keys()
    
    



In [3]:
def parseLine(characterName, line, file, lastSpeaker):
    non_actions = re.sub(r'[{\W}]', ' ', line.lower())
    non_actions = re.sub(r'[-{2,}]', '', non_actions)
    non_actions = re.sub(r'([a-z])\1{2,}', r'\1', non_actions)
    doc = nlp(line)
    lastToken = None
    index = 0
    foundSpokenTo = None
    
    ens = 'ensquanche' in line
    
    for token in doc:
       
        if lastToken != None and token.text in characters:
            if ens and lastToken != None:
                print(lastToken, token,lastToken.is_punct, token.is_punct)
            if token.is_punct == True and lastToken.text in characters:            
                foundSpokenTo = lastToken
            elif lastToken.is_punct == True and token.text in characters:
                foundSpokenTo = token
                    
        index += 1
        lastToken = token
    
#     print(line)
#     if foundSpokenTo != None:
#         print(characterName, '->', foundSpokenTo.text)
#     else:
#         print(characterName, '->', lastSpeaker)
#     print()
    return [characterName, lastSpeaker]
    


In [4]:
document = {}
files = os.listdir("./parsed")
for filename in files:
    print('Processing: ', filename)
    episodeName = filename[6:len(filename)-4];
    interaction = []
    lines = 0
    with open("./parsed/" + filename, "r") as f:             
        for line in f:            
            lines += 1
    
    with open("./parsed/" + filename, "r") as f:         
        lastSpeaker = ''
        for line in tqdm(f, total=lines): 
            if stripPersonnas == True:
                find = re.search(r"^([a-zA-Z][a-zA-Z \.]+).*\:(.*)", line) 
            else:
                find = re.search(r"^([a-z][a-z0-9 \(\)]+):(.*)", line)
            if find:
                name = find.group(1).strip()
                text = find.group(2).strip()
                if name not in ignore:
                    try:
                        actualName = replace[name]
                    except:
                        actualName = name
                    
                interaction.append(parseLine(actualName, text, filename[6:len(filename)-4], lastSpeaker))
                lastSpeaker = name
    document[episodeName] = interaction

  7%|▋         | 18/245 [00:00<00:01, 171.42it/s]

Processing:  011 - Ricksy Business.txt


  7%|▋         | 19/262 [00:00<00:01, 181.92it/s]

Processing:  021 - The Wedding Squanchers.txt


 21%|██        | 54/262 [00:00<00:01, 176.52it/s]

...

  6%|▌         | 21/376 [00:00<00:01, 206.11it/s]

Processing:  023 - Rickmancing the Stone.txt


  6%|▌         | 24/411 [00:00<00:01, 223.59it/s]

Processing:  012 - A Rickle in Time.txt


100%|██████████| 411/411 [00:02<00:00, 196.93it/s]


In [5]:
with open('interaction.json', 'w') as fp:
    json.dump(document, fp)